In [1]:
 def extract(dictionary,key):
    try:
        return dictionary[key]
    except:
        return 'none'

In [2]:
def trypull(list):
    if list == 'none':
        return list
    else:
        return list[0]

In [3]:
def bigactors(actors):
    actor_list = [i[0] for i in prolific]   
    count = 0
    for i in actors:
        if i in actor_list:
            count+=1
    return count

In [4]:
def convert_ytdate(dstring):
    if dstring == 'none':
        return 0
    elif dstring.find('days') > -1:
        return round(int(dstring.split(' ')[0]))
    elif dstring.find('months') > -1:
        d = int(dstring.split(' ')[0])
        return round(int(d*4.333*7))
    elif dstring.find('weeks') > -1:
        d = int(dstring.split(' ')[0])
        return round(int(d*7))
    elif dstring.find('years') > -1:
        d = int(dstring.split(' ')[0])
        return round(int(d*52*7))
    else: return 0

In [5]:
def append_crew(cast):
    for i in cast:
        try: castlist[i] += 1
        except: castlist[i] = 1


In [6]:
def rating_clean(rating):
    if rating[0] in ('n', 'M/PG', 'Open'):
        return 'none'
    else:
        return rating[0]
        

In [7]:
def split_rating(rating):
    try:
        return int(rating.split('%')[0])
    except:
        return 60

In [8]:
def ytviews(views):
    try:
        return int(views.split(' ')[0].replace(',',''))
    except:
        return 0

In [26]:
import pandas as pd
import numpy as np
with open('movies.json') as f:
    df = pd.read_json(f)

df = df.drop(['runtime','prodCountries','prodCompanies','keywords','comps','domesticReleases'], axis = 1)
cleaned = df.fillna('none')

cleaned['critics'] = cleaned['critics'].apply(lambda x: split_rating(x))
cleaned['audience'] = cleaned['audience'].apply(lambda x: split_rating(x))
cleaned['creative'] = cleaned['creative'].apply(lambda x: x[0])
cleaned['firstviews'] = cleaned['firstviews'].apply(lambda x: ytviews(x))
cleaned['firstdate'] = cleaned['firstdate'].apply(lambda x: convert_ytdate(x))
cleaned['genre'] = cleaned['genre'].apply(lambda x: x[0])
cleaned['budget'] = cleaned['budget'].apply(lambda x: int(x.replace(',','').replace('$','')))
cleaned['source'] = cleaned['source'].apply(lambda x: trypull(x))


cleaned['director'] = cleaned['crew'].apply(lambda x: extract(x, 'Director'))
cleaned['producer'] = cleaned['crew'].apply(lambda x: extract(x, 'Producer'))
cleaned['execprod'] = cleaned['crew'].apply(lambda x: extract(x,'Executive Producer'))
cleaned['scrwriter'] = cleaned['crew'].apply(lambda x: extract(x, 'Screenwriter'))

cleaned['released']  = pd.to_datetime(cleaned['released'], format='%m/%d/%Y')
cleaned['released'] = cleaned['released'].dt.quarter
cleaned['cast'] = cleaned['cast'].apply(lambda x: x.values())
cleaned['rating'] = cleaned['mpaaRating'].apply(lambda x: x[0])
cleaned['domestic'] = cleaned['domestic'].apply(lambda x: int(x.replace('$','').replace(',','')))
cleaned['worldwide'] = cleaned['worldwide'].apply(lambda x: int(x.replace('$','').replace(',','')))
cleaned['results'] = cleaned['results'].apply(lambda x: int(x.replace(',','')))
cleaned['franchise'] = cleaned['franchise'].apply(lambda x: trypull(x))
cleaned['mpaaRating'] = cleaned['mpaaRating'].apply(lambda x: rating_clean(x))
cleaned['internationalReleases'] = cleaned['internationalReleases'].apply(lambda x: len(x))


In [31]:
cleaned.producer.value_counts()

none                    2864
Jerry Bruckheimer         17
Albert R. Broccoli        15
Eric Fellner              12
Neal H. Moritz            12
Kevin Feige               12
Scott Rudin               12
Jack Giarraputo           12
Ryan Kavanaugh            10
Lauren Shuler Donner      10
Brian Grazer               9
Colin Wilson               9
Joel Silver                8
Ethan Coen                 8
Oren Peli                  8
Ian Bryce                  7
Sam Mercer                 7
Kathleen Kennedy           7
Barbara Broccoli           7
Lawrence Bender            7
Joe Roth                   7
Jack Rapke                 7
Anthony Katagas            6
Robert Lorenz              6
Jonathan Glickman          6
Scott Mosier               6
Frank Marshall             6
Mark Johnson               6
David Heyman               6
Michael Fottrell           6
                        ... 
Zack Snyder                1
Jennifer Schwalbach        1
Joe Hardesty               1
Ghislain Barro

In [11]:
gendummies = pd.get_dummies(cleaned['genre'], drop_first=True)
ratdummies = pd.get_dummies(cleaned['mpaaRating'],drop_first=True)
creatdummies = pd.get_dummies(cleaned['creative'], drop_first=True)
datdummies = pd.get_dummies(cleaned['released'], drop_first = True)
print type(gendummies), type(ratdummies), type(creatdummies),type(cleaned)
cleaned = pd.concat([cleaned, gendummies, ratdummies, creatdummies, datdummies], axis = 1)



<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [12]:
castlist = {}
castbymov = []
cleaned['cast'].apply(lambda x: append_crew(x));
for k,v in castlist.iteritems():
    tupe = (k,v)
    castbymov.append(tupe)
prolific = sorted(castbymov, key=lambda tup: tup[1], reverse=True)[:100]
cleaned['cast'] = cleaned['cast'].apply(lambda x:bigactors(x))

In [13]:
cleaned['origscreen'] = cleaned['source'].apply(lambda x: int(x=='Original Screenplay'))

In [14]:
to_model = cleaned.drop(['crew','franchise','source','title','mpaaRating','production','released','rank','producer','scrwriter','director','execprod','none','n','genre','Multiple Creative Types','Concert/Performance','creative','rating'], axis = 1)

In [15]:
to_model.columns.values

array([u'audience', u'budget', u'cast', u'critics', u'domestic',
       u'firstdate', u'firstviews', u'internationalReleases', u'results',
       u'worldwide', u'Adventure', u'Black Comedy', u'Comedy',
       u'Documentary', u'Drama', u'Horror', u'Musical', u'Romantic Comedy',
       u'Thriller/Suspense', u'Western', u'NC-17', u'Not Rated', u'PG',
       u'PG-13', u'R', u'Dramatization', u'Factual', u'Fantasy',
       u'Historical Fiction', u'Kids Fiction', u'Science Fiction',
       u'Super Hero', 2, 3, 4, 'origscreen'], dtype=object)

#### need to figure out how to handle producer, screenwriter, director, executiveproducer, & firstdate

In [16]:
import pandas as pd
import numpy as np
import math
from sklearn import cross_validation, linear_model, grid_search
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


def scatter_matrix(X):
    feature_count = len(X.columns)
    fig,ax = plt.subplots(ncols=feature_count,nrows=feature_count,figsize=(10*feature_count, 10*feature_count))

    for i,feature_i in enumerate(X):
        for j,feature_j in enumerate(X):
            ax[i][j].scatter(X[feature_i],X[feature_j])
            ax[i][j].set_xlabel('Feature ' + str(feature_i))
            ax[i][j].set_ylabel('Feature ' + str(feature_j))


/Users/R1/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [17]:
from __future__ import division

In [18]:
X,y = to_model.drop(['worldwide','domestic','internationalReleases', 'firstdate'], axis=1),to_model['worldwide']

X_train,X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size = 0.3)

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from sklearn import cross_validation, datasets, linear_model, grid_search, tree, preprocessing
from sklearn.tree import export_graphviz, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier,  RandomForestRegressor,  ExtraTreesClassifier
from sklearn.cross_validation import KFold

%matplotlib inline

In [ ]:
kf = KFold(len(X_train), n_folds=20, shuffle=True, random_state=0)


In [ ]:
lasso = linear_model.Lasso()

parameters = {'normalize':(True,False),
              'alpha':np.logspace(-2,50,100)}
grid_searcher = grid_search.GridSearchCV(lasso, parameters, cv=kf)

grid_searcher.fit(X_train, y_train)


In [ ]:
model = grid_searcher.best_estimator_
print(grid_searcher.best_params_)
print 'score', grid_searcher.score(X_test, y_test)
zip(X_train.columns,model.coef_)



In [ ]:
linreg = linear_model.LinearRegression()
var,targ = to_model.drop(['worldwide','domestic','internationalReleases', 'firstdate','Western','Black Comedy','Musical','Thriller/Suspense','NC-17','Not Rated','R','Factual','Super Hero'], axis=1),to_model['worldwide']
var_train,var_test, targ_train, targ_test = cross_validation.train_test_split(X,y,test_size = 0.3)
linreg.fit(var_train, targ_train)
print 'score', linreg.score(var_test,targ_test)
zip(var.columns, linreg.coef_)

In [ ]:
enet = linear_model.ElasticNet()

parameters = {'normalize':(True,False),
              'alpha':np.logspace(-2,30,50)}
grid_searcher = grid_search.GridSearchCV(enet, parameters, cv=kf)

grid_searcher.fit(X_train, y_train)

model = grid_searcher.best_estimator_
print(grid_searcher.best_params_)
print 'score', grid_searcher.score(X_test, y_test)
zip(X.columns,model.coef_)


In [20]:
from sklearn.ensemble import RandomForestRegressor

In [21]:
rf =RandomForestRegressor(n_estimators = 100, max_features = 7 )
rf.fit(X_train,y_train)
a  = X.columns.values
print "Score: " + str(rf.score(X_test,y_test))
sorted_features = zip(X.columns, rf.feature_importances_)
sorted_features.sort(key=lambda x: x[1], reverse=True)
for feature in sorted_features:
    print(feature)

Score: 0.63845577223
(u'budget', 0.36004288539665441)
(u'firstviews', 0.13380071377426975)
(u'audience', 0.087569060122586931)
(u'results', 0.074286846335061746)
(u'critics', 0.066046539498339149)
(u'cast', 0.058800594318821862)
(u'Adventure', 0.051322924485552396)
(u'Kids Fiction', 0.020665753310412757)
(u'PG-13', 0.018408680078643644)
(2, 0.016623120961614571)
(u'Science Fiction', 0.015363011049420955)
('origscreen', 0.012172486439953984)
(u'R', 0.011788991058389382)
(u'Fantasy', 0.010565550095769892)
(3, 0.0084856663320196056)
(4, 0.008189380800210316)
(u'PG', 0.0076102605918892926)
(u'Drama', 0.0074441115482380216)
(u'Comedy', 0.0055331533841327983)
(u'Super Hero', 0.0051517898237445677)
(u'Historical Fiction', 0.0037744035181258707)
(u'Thriller/Suspense', 0.0036056440299955432)
(u'Dramatization', 0.0030201048041299861)
(u'Horror', 0.0022316581656789077)
(u'Romantic Comedy', 0.0021329878404831599)
(u'Not Rated', 0.0018282165068152028)
(u'Musical', 0.0017569342914217801)
(u'Western'

In [22]:
y_train

751              0
533        5892466
745        4600000
4518     442169052
4932     132999811
2194      85512300
1764      76200721
1816       4313644
4611     162870525
3296      27866213
1297             0
2160      12516222
724         842693
412          58163
897         593933
4040      59551283
59               0
4673       9103630
2591      54294169
5068      20600000
3795      17378193
3331      27257061
1603       5010509
4357             0
1699             0
3394     139109346
4225     257805243
814        2077844
1067         19875
4253     355612291
           ...    
3696      54768418
1176       3858964
2949      79114085
1579      44103982
1036      19184820
1894        109383
3584      79915361
2273      18733666
5107     542742489
3768     160000000
1998      25697647
3235      97030468
1674        555335
1459         15091
878         500296
1807         14443
4477     101636047
4838    1141408667
2871      32632668
311              0
1704        446438
1209      57

In [ ]:
rf.score(X_test,y_test)


In [ ]:
X_train.columns

In [ ]:
import statsmodels.api as sm

In [ ]:
olsm = sm.OLS(y_train, X_train)
results = olsm.fit()
results.rsquared

In [23]:
df.columns

Index([u'audience', u'budget', u'cast', u'creative', u'crew', u'critics',
       u'domestic', u'firstdate', u'firstviews', u'franchise', u'genre',
       u'internationalReleases', u'mpaaRating', u'production', u'rank',
       u'released', u'results', u'source', u'title', u'worldwide'],
      dtype='object')

In [25]:
df.crew

0       {u'Casting (Australia)': u'Ann Robinson', u'Re...
1       {u'Stunt Coordinator': u'Damon Caro', u'Superv...
2       {u'Stunt Coordinator': u'Spiro Razatos', u'Re-...
3       {u'Second Assistant Director-Second Unit': u'T...
4       {u'Stunt Coordinator': u'Scott Rogers', u'Cost...
5       {u'Animation Supervisor': u'Clay Kaytis', u'As...
6       {u'Stunt Coordinator': u'Tom Struthers', u'Bas...
7       {u'ILM Visual Effects Executive Producer': u'J...
8       {u'Stunt Coordinator': u'Tom Struthers', u'Re-...
9       {u'Stunt Coordinator': u'Gary Powell', u'Re-re...
10      {u'Stunt Coordinator': u'George Marshall Ruge'...
11      {u'Re-recording Mixer': u'Christopher Scarabos...
12      {u'Based on Characters Created By': u'Michael ...
13      {u'Stunt Coordinator': u'James Armstrong', u'R...
14      {u'Producer': u'Philip Steuer', u'Visual Effec...
15      {u'Producer': u'Peter Safran', u'Sound Designe...
16                                                     {}
17            